### --- Step 1: Imports ---

In [ ]:
from functions.llm_chain import generate_response_openai

In [ ]:
import hopsworks
import exclude.key
from openai import OpenAI
import os
import getpass
import joblib

# Hopsworks
HOPSWORKS_API_KEY = exclude.key.HOPSWORKS_API_KEY
OPENAI_API_KEY = exclude.key.OPENAI_API_KEY

FEATURE_GROUP_NAME = "bars_near_london_bridge"
FEATURE_VIEW_NAME = "busyness_feature_view"
FEATURE_GROUP_VERSION = 3
FEATURE_VIEW_VERSION = 4

project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

In [ ]:
fg = fs.get_feature_group(
    name=FEATURE_GROUP_NAME, 
    version=FEATURE_GROUP_VERSION,
)
fg

## -- Step 2 Retrieve Feature View -- 

In [ ]:
# Retrieve the 'busyness_feature_view'
feature_view = fs.get_feature_view(
    name=FEATURE_VIEW_NAME,
    version=FEATURE_VIEW_VERSION,
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)
feature_view

In [ ]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'busyness_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="xgboost_busyness",
    version=3,
)
retrieved_model

In [ ]:
model_dir = "busyness_model"

# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'busyness_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="xgboost_busyness",
    version=3,
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()
saved_model_dir = "./"

# Load the XGBoost regressor model and label encoder from the saved model directory
model_busyness = joblib.load(saved_model_dir + f"{model_dir}/xgboost_busyness.pkl")
# encoder = joblib.load(saved_model_dir + "/label_encoder.pkl")

# Display the retrieved XGBoost regressor model
model_busyness


## -- Step 3 Ask the Question! -- 

In [ ]:

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [ ]:
QUESTION = "What will be the best bar to go in an hour from now next to London Bridge?"

response = generate_response_openai(   
    QUESTION,
    feature_view,
    model_busyness,
    client,
    verbose=True,
)
print(response)